In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [25]:
pip install torch transformers datasets peft accelerate scikit-learn evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [17]:
import torch
import transformers
from transformers import AutoTokenizer, BertForSequenceClassification
from peft import LoraConfig, get_peft_model, TaskType
from datasets import load_dataset

In [8]:
dataset = load_dataset("ag_news")

README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [9]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)
print(tokenized_datasets)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7600
    })
})


In [12]:
print(tokenized_datasets["train"][0])

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'label': 2, 'input_ids': [101, 6250, 1457, 119, 10169, 140, 9598, 4388, 14000, 1103, 2117, 113, 11336, 27603, 114, 11336, 27603, 118, 6373, 118, 18275, 1116, 117, 6250, 1715, 112, 188, 173, 11129, 1979, 165, 1467, 1104, 18737, 118, 172, 5730, 4724, 117, 1132, 3195, 2448, 1254, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [14]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [18]:
# LoRa
config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # Sequence classification
    inference_mode=False,  # Training mode
    r=8,  # Low-rank dimension
    lora_alpha=32,  # Scaling factor for LoRA
    lora_dropout=0.1  # Dropout for LoRA layers
)

In [19]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-cased', 
    num_labels=4
)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
model = get_peft_model(model, config)

In [23]:
model.print_trainable_parameters()

trainable params: 297,988 || all params: 108,611,336 || trainable%: 0.2744


In [26]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [34]:
from transformers import TrainingArguments, Trainer


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


training_args = TrainingArguments(
    output_dir="outputs", 
    max_steps=500,
    learning_rate=2e-4,
    auto_find_batch_size=True,
    logging_steps=1
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics    
)


max_steps is given, it will override any value given in num_train_epochs


In [35]:
trainer.train()

Step,Training Loss
1,1.019700
2,0.926900
3,0.762800
4,0.766700
5,0.712500
6,1.266400
7,0.925800
8,0.654100
9,0.829100
10,0.898400


TrainOutput(global_step=500, training_loss=0.4429442072436213, metrics={'train_runtime': 8300.398, 'train_samples_per_second': 0.482, 'train_steps_per_second': 0.06, 'total_flos': 1056124796928000.0, 'train_loss': 0.4429442072436213, 'epoch': 4.0})

In [36]:
torch.save(model.state_dict(), 'lora_bert_classifier.pt')

In [1]:
import torch
from transformers import AutoTokenizer, AutoConfig, BertForSequenceClassification
from peft import LoraConfig, get_peft_model

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = BertForSequenceClassification.from_pretrained(
    'bert-base-cased', 
    device_map='auto'
)

# model = BertForSequenceClassification.from_pretrained(
#     "gpt2",
#     device_map='auto',
# )

config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # Sequence classification
    inference_mode=False,  # Training mode
    r=8,  # Low-rank dimension
    lora_alpha=32,  # Scaling factor for LoRA
    lora_dropout=0.1  # Dropout for LoRA layers
)

model = get_peft_model(model, config)
model = model.to(device)
model.load_state_dict(torch.load("lora_bert_classifier.pt", map_location=device))

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
# tokenizer.pad_token = tokenizer.eos_token

with torch.no_grad():
    batch = tokenizer("“Life is like a box of chocolates, you never know what you are gonna get” ->: ", return_tensors='pt').to(device)
    output_tokens = model.generate(**batch, max_new_tokens=15)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

ModuleNotFoundError: No module named 'peft'